In [ ]:
#default_exp vision.core

In [ ]:
#export
from fastai_local.imports import *
from fastai_local.test import *
from fastai_local.core import *
from fastai_local.data.pipeline import *
from fastai_local.data.core import *
from fastai_local.data.external import *

from PIL import Image

In [ ]:
#export
_all_ = ['Image']

# Core vision
> Basic image opening/processing functionality

In [ ]:
path = untar_data(URLs.MNIST_TINY)
fns = get_image_files(path)
fn = fns[0]
fn

PosixPath('/home/ubuntu/git/fastai_docs/dev/data/mnist_tiny/test/4605.png')

In [ ]:
img = Image.open(fn)

In [ ]:
#export
def image_convert(img, mode='RGB'):
    "Convert `img` to `mode`"
    return img.convert(mode)

In [ ]:
#export
def ImageConverter(mode='RGB'): return partialler(image_convert, mode=mode)

In [ ]:
f = ImageConverter('RGB')
test_eq(f(img).mode, 'RGB')

In [ ]:
#export
def image_resize(img, size, resample=Image.BILINEAR):
    "Resize image to `size` using `resample"
    return img.resize(size, resample=resample)
image_resize.order=10

In [ ]:
#export
def ImageResizer(size, resample=Image.BILINEAR):
    if not is_listy(size): size=(size,size)
    return partialler(image_resize, size=size, resample=resample)

In [ ]:
f = ImageResizer(14)
test_eq(f(img).size, (14,14))
img

In [ ]:
# export
def image2byte(img):
    "Transform image to byte tensor in `c*h*w` dim order."
    res = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes()))
    w,h = img.size
    return res.view(h,w,-1).permute(2,0,1)

In [ ]:
# export
def unpermute_image(img):
    "Convert `c*h*w` dim order to `h*w*c` (or just `h*w` if 1 channel)"
    return img[0] if img.shape[0] == 1 else img.permute(1,2,0)

In [ ]:
# export
def ImageToByteTensor():
    "Transform image to byte tensor in `c*h*w` dim order."
    return Transform(image2byte, decodes=unpermute_image, order=15)

In [ ]:
tfm = ImageToByteTensor()
t = tfm(img)
test_eq(t.shape, (1,28,28))
test_eq(tfm.decode(t).shape, (28,28))

The transformation to tensors is done in two steps just in case you want to apply transforms to byte tensors or want to use the GPU for converting to floats. The permutation of axes needs to be reversed for display, so we define `decodes`.

In [ ]:
# export
class ByteToFloatTensor(Transform):
    "Transform image to float tensor, optionally dividing by 255 (e.g. for images)."
    order=20 #Need to run after CUDA if on the GPU
    def __init__(self, div=True): self.div = div
    def encodes(self, x): return x.float().div_(255.) if self.div else x.float()
    def decodes(self, x): return x.clamp(0., 1.) if self.div else x

In [ ]:
tfm = ByteToFloatTensor()
ft = tfm(t)
test_eq(ft.max(), 1.)

## Export -

In [ ]:
#hide
from fastai_local.notebook.export import notebook2script
notebook2script(all_fs=True)

JSONDecodeError: Expecting ',' delimiter: line 110 column 3 (char 4490)